https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #rescale=1./255,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='wrap'
)


In [3]:
#img = load_img('data/test/crack/img_equalized_5526_856.png', grayscale=True)  # this is a PIL image
#img

In [4]:
#x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#x.shape

In [5]:
#x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
#x = np.expand_dims(x, axis=0)
#x.shape


In [6]:
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
#i = 0
#for batch in datagen.flow(x, batch_size=1, 
#                          save_to_dir='preview', save_prefix='img15', save_format='png'):
#    i += 1
#    if i > 10:
#        break  # otherwise the generator would loop indefinitely

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [8]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [9]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [10]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [12]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [13]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        9248      
__________

In [14]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='wrap'
)


In [15]:
test_datagen = ImageDataGenerator(
    rescale=1./255)


In [16]:
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(100, 100),  # all images will be resized to 150x150
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels



Found 7897 images belonging to 2 classes.


In [ ]:
# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(100, 100),
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')

Found 4557 images belonging to 2 classes.


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=200,
        validation_data=test_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  #

Epoch 1/200
125/125 [==============================] - 27s - loss: 0.5778 - acc: 0.7170 - val_loss: 0.7182 - val_acc: 0.6138
Epoch 2/200
125/125 [==============================] - 27s - loss: 0.5189 - acc: 0.7745 - val_loss: 1.6049 - val_acc: 0.3962
Epoch 3/200
125/125 [==============================] - 32s - loss: 0.4540 - acc: 0.8150 - val_loss: 2.9061 - val_acc: 0.3688
Epoch 4/200
125/125 [==============================] - 33s - loss: 0.4282 - acc: 0.8208 - val_loss: 5.2789 - val_acc: 0.2587
Epoch 5/200
125/125 [==============================] - 35s - loss: 0.3744 - acc: 0.8460 - val_loss: 4.5513 - val_acc: 0.2961
Epoch 6/200
125/125 [==============================] - 33s - loss: 0.3761 - acc: 0.8460 - val_loss: 4.9053 - val_acc: 0.3187
Epoch 7/200
125/125 [==============================] - 32s - loss: 0.3555 - acc: 0.8530 - val_loss: 5.8816 - val_acc: 0.2863
Epoch 8/200
125/125 [==============================] - 32s - loss: 0.3477 - acc: 0.8612 - val_loss: 4.7731 - val_acc: 0.3262


125/125 [==============================] - 31s - loss: 0.2047 - acc: 0.9280 - val_loss: 8.9672 - val_acc: 0.2737
Epoch 67/200
125/125 [==============================] - 31s - loss: 0.2011 - acc: 0.9295 - val_loss: 8.8442 - val_acc: 0.3413
Epoch 68/200
125/125 [==============================] - 31s - loss: 0.1909 - acc: 0.9306 - val_loss: 8.3117 - val_acc: 0.3675
Epoch 69/200
125/125 [==============================] - 31s - loss: 0.2164 - acc: 0.9260 - val_loss: 8.2638 - val_acc: 0.3600
Epoch 70/200
125/125 [==============================] - 31s - loss: 0.2014 - acc: 0.9315 - val_loss: 7.7021 - val_acc: 0.3925
Epoch 71/200
125/125 [==============================] - 31s - loss: 0.2249 - acc: 0.9235 - val_loss: 7.8880 - val_acc: 0.3800
Epoch 72/200
125/125 [==============================] - 32s - loss: 0.1805 - acc: 0.9376 - val_loss: 7.9284 - val_acc: 0.3814
Epoch 73/200
125/125 [==============================] - 31s - loss: 0.1993 - acc: 0.9360 - val_loss: 8.4119 - val_acc: 0.3750
Epoch